## Objectif
Ce notebook démontre comment utiliser `morpho-analytics` pour suivre des structures biologiques synthétiques.

### Imports

In [ ]:
from pathlib import Path
import json

import numpy as np
from IPython.display import Image, display

from morpho_analytics.track import track, metrics, save_overlay_png

### Charger ou générer la série synthétique

In [ ]:
data_dir = Path.cwd()
synth_path = data_dir / "synth.npy"

if not synth_path.exists():
    T, H, W, r = 20, 128, 128, 10.0
    ys, xs = np.mgrid[0:H, 0:W]
    series = np.zeros((T, H, W), dtype=np.float32)
    cx0, cy0, vx, vy = 20.0, 20.0, 3.0, 2.0
    for t in range(T):
        cx, cy = cx0 + vx * t, cy0 + vy * t
        d2 = (xs - cx) ** 2 + (ys - cy) ** 2
        series[t] = np.exp(-d2 / (2.0 * (r ** 2)))
    series -= series.min()
    m = series.max()
    if m > 0:
        series /= m
    np.save(synth_path, series.astype(np.float32))

series = np.load(synth_path, allow_pickle=False)
series.shape

### Suivre la série

In [ ]:
tracks = track(series, threshold=None, connectivity=4, min_area=5)
tracks

### Calculer les métriques

In [ ]:
report = metrics(tracks)
print(json.dumps(report, indent=2))

### Sauvegarder le rapport JSON

In [ ]:
report_path = Path.cwd() / "bio_report.json"
report_path.write_text(json.dumps(report, indent=2))
report_path

### Générer et afficher l'overlay

In [ ]:
overlay_path = Path.cwd() / "bio_overlay.png"
save_overlay_png(series, tracks.labels, str(overlay_path))
display(Image(filename=str(overlay_path)))